#  Полный отчёт: Работа с изображениями в задачах искусственного интеллекта  
*Для подготовки к чемпионату  (на примере датасета «Dog Breed Image Dataset»)*


## 🔹 Способы организации датасета

Перед тем как приступить к загрузке и обработке изображений, важно понять, **как организованы данные**. От этого зависит способ их считывания и дальнейшей обработки. Существует два основных подхода, которые применяются на практике.


###  Способ 1: Каждая папка — отдельный класс (основной формат)

Это **наиболее распространённый и рекомендуемый способ** организации размеченных изображений. 

**Структура файловой системы выглядит следующим образом:**
```
dog_breeds/
├── Labrador/
│   ├── 001.jpg
│   ├── 002.jpg
│   └── ...
├── Beagle/
│   ├── 101.jpg
│   └── ...
├── Poodle/
│   └── ...
└── ...
```

В этой структуре:
- Название каждой подпапки (например, `Labrador`) **является меткой класса**.
- Все изображения, находящиеся внутри этой подпапки, **относятся к данному классу**.
- Нет необходимости хранить отдельный файл с разметкой — структура папок сама по себе содержит всю необходимую информацию.

**Преимущества этого подхода:**
- Простота и интуитивная понятность.
- Прямая поддержка ведущими библиотеками машинного обучения, такими как PyTorch (`torchvision.datasets.ImageFolder`), TensorFlow (`tf.keras.utils.image_dataset_from_directory`) и YOLO (Ultralytics).
- Минимальный риск ошибок при загрузке данных.


###  Способ 2: Метка класса указана в имени файла

В некоторых случаях, особенно в устаревших или кастомных датасетах, все изображения могут находиться в **одной общей папке**, а метка класса **кодируется непосредственно в имени файла**.

**Пример структуры:**
```
images/
├── Labrador_001.jpg
├── Beagle_002.jpg
├── Poodle_003.jpg
└── ...
```

Здесь:
- Класс (например, `Labrador`) является **первой частью имени файла**, отделённой от остального идентификатора символом подчёркивания `_`.
- Чтобы определить метку, необходимо **проанализировать имя файла** и извлечь из него соответствующую часть.


In [11]:
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Для CNN (нейросети)
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms


## 🔹  Загрузка данных

### Загрузка по папкам (основной способ)
Эта функция автоматически определяет класс по названию папки и не требует ручного анализа имён файлов

In [12]:
#(класс = имя папки)

DATASET_DIR = Path("dataset")

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

rows = []
for class_dir in sorted([p for p in DATASET_DIR.iterdir() if p.is_dir()]):
    label = class_dir.name
    for img_path in class_dir.rglob("*"):
        if img_path.suffix.lower() in IMG_EXTS:
            rows.append({"path": str(img_path), "label": label})

df = pd.DataFrame(rows)
print("Всего изображений:", len(df))
print("Классы:", df["label"].unique())
print(df["label"].value_counts())


Всего изображений: 967
Классы: ['Beagle' 'Boxer' 'Bulldog' 'Dachshund' 'German_Shepherd'
 'Golden_Retriever' 'Labrador_Retriever' 'Poodle' 'Rottweiler'
 'Yorkshire_Terrier']
label
Beagle                100
Boxer                 100
Bulldog               100
Poodle                100
Yorkshire_Terrier     100
Dachshund              96
German_Shepherd        96
Labrador_Retriever     95
Golden_Retriever       91
Rottweiler             89
Name: count, dtype: int64


### Загрузка по именам файлов
Для менее распространённого формата, где метка содержится в имени файла, реализуем отдельную функцию:

In [13]:
DATASET_DIR_FLAT = Path("dataset")  

def label_from_filename(filename: str) -> str:
    stem = Path(filename).stem
    return re.split(r"[_-]", stem)[0]

rows2 = []
for img_path in DATASET_DIR_FLAT.rglob("*"):
    if img_path.is_file() and img_path.suffix.lower() in IMG_EXTS:
        rows2.append({"path": str(img_path), "label": label_from_filename(img_path.name)})

df2 = pd.DataFrame(rows2)
print("Всего изображений:", len(df2))
print("Классы:", df2["label"].unique())
print(df2["label"].value_counts())


Всего изображений: 967
Классы: ['Rottweiler' 'Dachshund' 'Yorkshire Terrier' 'Beagle' 'Golden Retriever'
 'Poodle' 'Labrador Retriever' 'German Shepherd' 'Boxer' 'Bulldog']
label
Yorkshire Terrier     100
Beagle                100
Poodle                100
Boxer                 100
Bulldog               100
Dachshund              96
German Shepherd        96
Labrador Retriever     95
Golden Retriever       91
Rottweiler             89
Name: count, dtype: int64


## 🔹  Предобработка изображений

После загрузки изображение необходимо **подготовить к подаче в модель**. Все изображения должны быть:

- Преобразованы в единый цветовой формат (обычно RGB),
- Приведены к одинаковому размеру (например, 224×224 пикселей),
- Нормализованы (значения пикселей приведены к диапазону [0, 1] или [-1, 1]).

Это необходимо, потому что нейронные сети **не могут работать с изображениями разного размера**, а нормализация **ускоряет и стабилизирует обучение**.


In [14]:
# функция предобработки в numpy (RGB + resize + нормализация)

TARGET_SIZE = (128, 128)

def preprocess_pil_to_numpy(img_path: str, target_size=TARGET_SIZE) -> np.ndarray:
    img = Image.open(img_path).convert("RGB")
    img = img.resize(target_size)
    arr = np.array(img, dtype=np.float32) / 255.0  # нормализация в [0, 1]
    return arr  # shape: (H, W, 3)

# тест
sample_path = df["path"].iloc[0]
x = preprocess_pil_to_numpy(sample_path)


In [15]:
#transforms для PyTorch

torch_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(), 
    # Можно добавить нормализацию mean/std, но для простого старта достаточно ToTensor()
])


## 🔹  Балансировка классов (чтобы “примеров было +- поровну”)

В реальных датасетах часто встречаются **дисбалансированные классы**: одни породы представлены сотнями изображений, другие — всего десятком. Это приводит к тому, что модель **«учится игнорировать» редкие классы**.

###  Для RandomForest (sklearn):

Добавьте параметр `class_weight="balanced"` при создании модели — это автоматически взвесит ошибки на редких классах сильнее.

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)

### Балансировка “чтобы поровну” = WeightedRandomSampler
Чтобы классы встречались примерно одинаково в обучении — делай семплинг через WeightedRandomSampler.

In [17]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler

# 1. Определяем трансформации
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root="dataset", transform=train_transform)

# 3. Балансировка через семплер
targets = torch.tensor(dataset.targets) 
class_count = torch.bincount(targets)
class_weight = 1.0 / class_count.float()
sample_weight = class_weight[targets]

sampler = WeightedRandomSampler(
    weights=sample_weight,
    num_samples=len(dataset),
    replacement=True
)

# 4. Создаём DataLoader
train_loader = DataLoader(dataset, batch_size=32, sampler=sampler, num_workers=2)

print(f" Датасет загружен: {len(dataset)} изображений, {len(dataset.classes)} классов")
print(f" DataLoader с балансировкой готов!")

 Датасет загружен: 967 изображений, 10 классов
 DataLoader с балансировкой готов!


### Для YOLOv8-cls (Ultralytics)
В YOLOv8 классификация обычно “ест” структуру папок, а балансировку чаще решают подготовкой данных (оверсэмплинг редких классов копированием/доп-аугментациями) или своими загрузчиками, потому что “скейлер” как в sklearn там не используется — нормализация/препроцессинг спрятаны внутри пайплайна.


## 🔹 Аугментация изображений 

Аугментация — это **ключевой приём** в компьютерном зрении, который позволяет искусственно увеличить объём обучающих данных и повысить устойчивость модели к вариациям во входных данных.

Цель аугментации — **смоделировать различные условия съёмки**, такие как:
- Другой ракурс (поворот, отражение),
- Изменённое освещение (яркость, контраст),
- Частичное закрытие объекта (случайное стирание),
- Отсутствие цветовой информации (чёрно-белый режим).

Это помогает модели **не переобучаться** на конкретные примеры, а **научиться распознавать суть объекта**.

### Основные виды аугментации

| Тип аугментации | Описание | Пример использования |
|------------------|--------|----------------------|
| **Горизонтальное отражение** | Зеркальное отображение изображения по вертикальной оси. | Полезно для симметричных объектов (лица, собаки, автомобили). |
| **Поворот** | Поворот изображения на случайный угол в заданном диапазоне. | Имитирует изменение угла съёмки. |
| **Изменение цвета** | Случайное изменение яркости, контраста, насыщенности и оттенка. | Моделирует разные условия освещения. |
| **Чёрно-белый режим** | Случайное преобразование изображения в градации серого. | Повышает устойчивость к цветовым артефактам. |
| **Случайное стирание (Random Erasing)** | Закрашивание случайного прямоугольного участка изображения чёрным цветом. | Имитирует потерю части объекта (например, за кустом). |

### Полный набор трансформаций для обучения

Для реализации аугментации мы будем использовать библиотеку `torchvision.transforms`, которая предоставляет удобные и эффективные инструменты.

# Получаем в итоге большой расширенный датасэт dataset_aug

In [18]:
from pathlib import Path
import shutil
import random
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split

TARGET_SIZE = (128, 128)

DATASET_DIR = Path("dataset")
TRAIN_AUG_DIR = Path("dataset_train_aug")   # train = оригиналы train + их аугменты
TEST_CLEAN_DIR = Path("dataset_test_clean") # test = только оригиналы test
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# 1) Индексация оригиналов
rows = []
for class_dir in sorted([p for p in DATASET_DIR.iterdir() if p.is_dir()]):
    label = class_dir.name
    for img_path in class_dir.rglob("*"):
        if img_path.suffix.lower() in IMG_EXTS:
            rows.append({"path": str(img_path), "label": label})
df = pd.DataFrame(rows)

# 2) Split оригиналов
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)

def reset_dir(p: Path):
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

def copy_split(df_part: pd.DataFrame, out_root: Path):
    for _, row in df_part.iterrows():
        src = Path(row["path"])
        label = row["label"]
        (out_root / label).mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, out_root / label / src.name)

reset_dir(TRAIN_AUG_DIR)
reset_dir(TEST_CLEAN_DIR)

# 3) Кладём оригиналы: train -> TRAIN_AUG_DIR, test -> TEST_CLEAN_DIR
copy_split(train_df, TRAIN_AUG_DIR)
copy_split(test_df, TEST_CLEAN_DIR)

# 4) Аугментации (ТОЛЬКО ДЛЯ TRAIN)
augment = transforms.Compose([
    transforms.Resize(TARGET_SIZE),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
    transforms.RandomGrayscale(p=0.25),  # 25% будут ч/б
])

def add_black_square(pil_img: Image.Image, max_frac=0.35) -> Image.Image:
    img = pil_img.copy()
    w, h = img.size
    rw = int(w * random.uniform(0.1, max_frac))
    rh = int(h * random.uniform(0.1, max_frac))
    x0 = random.randint(0, max(0, w - rw))
    y0 = random.randint(0, max(0, h - rh))
    arr = np.array(img)
    arr[y0:y0+rh, x0:x0+rw, :] = 0
    return Image.fromarray(arr)

def augment_train_only(train_df: pd.DataFrame, copies_per_image=2):
    for _, row in train_df.iterrows():
        src_path = Path(row["path"])
        label = row["label"]
        out_class_dir = TRAIN_AUG_DIR / label

        for k in range(copies_per_image):
            img = Image.open(src_path).convert("RGB")
            img = augment(img)
            img = add_black_square(img)
            img.save(out_class_dir / f"{src_path.stem}_aug{k}{src_path.suffix}")

augment_train_only(train_df, copies_per_image=2)

print("Train (with aug):", TRAIN_AUG_DIR)
print("Test (clean):", TEST_CLEAN_DIR)
print("Train orig:", len(train_df), "Test:", len(test_df))


Train (with aug): dataset_train_aug
Test (clean): dataset_test_clean
Train orig: 773 Test: 194


In [19]:
# индексируем расширенный датасет

rows_aug = []
for class_dir in sorted([p for p in TRAIN_AUG_DIR.iterdir() if p.is_dir()]):
    label = class_dir.name
    for img_path in class_dir.rglob("*"):
        if img_path.suffix.lower() in IMG_EXTS:
            rows_aug.append({"path": str(img_path), "label": label})

df_aug = pd.DataFrame(rows_aug)
print("Всего изображений после аугментации:", len(df_aug))
print(df_aug["label"].value_counts())


Всего изображений после аугментации: 2319
label
Beagle                240
Boxer                 240
Bulldog               240
Poodle                240
Yorkshire_Terrier     240
Dachshund             231
German_Shepherd       228
Labrador_Retriever    228
Golden_Retriever      219
Rottweiler            213
Name: count, dtype: int64


## 🔹 Train/test split + балансировка классов
Разделение (stratify)
Зачем: чтобы доли классов в train/test были похожи.

Балансировка для обучения
Есть 2 популярных подхода:

WeightedRandomSampler (для DataLoader в PyTorch) — при обучении чаще подбирать редкие классы.

class_weight (для sklearn моделей) — штрафовать ошибки по редким классам сильнее.

In [20]:
train_df, test_df = train_test_split(
    df_aug,
    test_size=0.2,
    random_state=42,
    stratify=df_aug["label"]
)

print("Train:", len(train_df), "Test:", len(test_df))
print("Train class counts:\n", train_df["label"].value_counts())
print("Test class counts:\n", test_df["label"].value_counts())

Train: 1855 Test: 464
Train class counts:
 label
Yorkshire_Terrier     192
Beagle                192
Bulldog               192
Boxer                 192
Poodle                192
Dachshund             185
Labrador_Retriever    182
German_Shepherd       182
Golden_Retriever      175
Rottweiler            171
Name: count, dtype: int64
Test class counts:
 label
Beagle                48
Yorkshire_Terrier     48
Poodle                48
Bulldog               48
Boxer                 48
Labrador_Retriever    46
Dachshund             46
German_Shepherd       46
Golden_Retriever      44
Rottweiler            42
Name: count, dtype: int64


## 🔹  Популярные модели для задач компьютерного зрения

| Задача | Популярные модели | Когда использовать |
|--------|------------------|-------------------|
| **Классификация** | ResNet, EfficientNet, Vision Transformer (ViT), **YOLOv8-cls** | Определение объекта на всём изображении |
| **Обнаружение объектов** | YOLOv8, Faster R-CNN, SSD | Нужны координаты и класс каждого объекта |
| **Сегментация** | U-Net, Mask R-CNN, YOLOv8-seg | Требуется точное выделение границ объекта |
| **Генерация изображений** | Stable Diffusion, GAN | Создание новых изображений |
| **Повышение разрешения** | ESRGAN, SRCNN | Улучшение качества изображений |

> Для задач классификации на чемпионате **рекомендуется начинать с YOLOv8-cls или EfficientNet-B0**.


### 🔹 Модель 1: RandomForest по «числам» (пиксели -> вектор)
Идея: превратить картинку 
128×128×3 в вектор длины 128∗128∗3 и обучить RandomForest. 

In [21]:
# Ячейка 5A: кодируем классы числами
classes = sorted(train_df["label"].unique())
class_to_id = {c: i for i, c in enumerate(classes)}
id_to_class = {i: c for c, i in class_to_id.items()}

train_df = train_df.copy()
test_df = test_df.copy()
train_df["y"] = train_df["label"].map(class_to_id)
test_df["y"] = test_df["label"].map(class_to_id)

print(class_to_id)


{'Beagle': 0, 'Boxer': 1, 'Bulldog': 2, 'Dachshund': 3, 'German_Shepherd': 4, 'Golden_Retriever': 5, 'Labrador_Retriever': 6, 'Poodle': 7, 'Rottweiler': 8, 'Yorkshire_Terrier': 9}


In [22]:
# делаем признаки для sklearn 

def build_X_y(df_in: pd.DataFrame, target_size=TARGET_SIZE):
    X_list, y_list = [], []
    for _, row in df_in.iterrows():
        arr = preprocess_pil_to_numpy(row["path"], target_size=target_size)  # (H,W,3) float32 [0..1]
        feat = arr.reshape(-1) 
        X_list.append(feat)
        y_list.append(int(row["y"]))
    return np.stack(X_list), np.array(y_list)

X_train, y_train = build_X_y(train_df)
X_test, y_test = build_X_y(test_df)

print(X_train.shape, y_train.shape)


(1855, 49152) (1855,)


In [23]:
# RandomForest обучение

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"  # балансировка внутри модели
)
rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred_rf))
print("F1 macro:", f1_score(y_test, pred_rf, average="macro"))
print(classification_report(y_test, pred_rf, target_names=classes))


Accuracy: 0.8836206896551724
F1 macro: 0.883577743138486
                    precision    recall  f1-score   support

            Beagle       0.93      0.85      0.89        48
             Boxer       0.81      0.90      0.85        48
           Bulldog       0.98      0.88      0.92        48
         Dachshund       0.76      0.96      0.85        46
   German_Shepherd       0.88      0.91      0.89        46
  Golden_Retriever       0.86      0.84      0.85        44
Labrador_Retriever       0.98      0.91      0.94        46
            Poodle       0.97      0.73      0.83        48
        Rottweiler       0.83      0.90      0.86        42
 Yorkshire_Terrier       0.92      0.96      0.94        48

          accuracy                           0.88       464
         macro avg       0.89      0.88      0.88       464
      weighted avg       0.89      0.88      0.88       464



## 🔹 Модель 2: Logistic Regression (ещё один baseline по числам)

In [24]:
#  LogisticRegression (часто сильнее, чем кажется, но требует больше итераций)

lr = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    n_jobs=None
)
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred_lr))
print("F1 macro:", f1_score(y_test, pred_lr, average="macro"))
print(classification_report(y_test, pred_lr, target_names=classes))


Accuracy: 0.7931034482758621
F1 macro: 0.7937177652126735
                    precision    recall  f1-score   support

            Beagle       0.84      0.75      0.79        48
             Boxer       0.78      0.83      0.81        48
           Bulldog       0.82      0.75      0.78        48
         Dachshund       0.65      0.85      0.74        46
   German_Shepherd       0.89      0.87      0.88        46
  Golden_Retriever       0.83      0.89      0.86        44
Labrador_Retriever       0.73      0.78      0.76        46
            Poodle       0.74      0.71      0.72        48
        Rottweiler       0.85      0.69      0.76        42
 Yorkshire_Terrier       0.87      0.81      0.84        48

          accuracy                           0.79       464
         macro avg       0.80      0.79      0.79       464
      weighted avg       0.80      0.79      0.79       464



## 🔹 YOLO для классификации (YOLO-cls)


Подготовка структуры из dataset_aug
Сейчас: dataset_aug/Beagle/*.jpg, dataset_aug/Boxer/*.jpg и т.п.
YOLO-cls ожидает так:
dataset_yolo_cls/train/<class>/*.jpg и dataset_yolo_cls/val/<class>/*.jpg (и опционально test).
​

In [25]:
import shutil
from pathlib import Path

SRC_TRAIN = Path("dataset_train_aug")
SRC_VAL = Path("dataset_test_clean")   # чистая проверка
DST = Path("dataset_yolo_cls")

TRAIN_DIR = DST / "train"
VAL_DIR = DST / "val"

if DST.exists():
    shutil.rmtree(DST)
TRAIN_DIR.mkdir(parents=True, exist_ok=True)
VAL_DIR.mkdir(parents=True, exist_ok=True)

def copy_tree(src_root: Path, dst_root: Path):
    for class_dir in src_root.iterdir():
        if not class_dir.is_dir():
            continue
        out = dst_root / class_dir.name
        out.mkdir(parents=True, exist_ok=True)
        for p in class_dir.glob("*"):
            if p.is_file():
                shutil.copy2(p, out / p.name)

copy_tree(SRC_TRAIN, TRAIN_DIR)
copy_tree(SRC_VAL, VAL_DIR)

print("YOLO dataset ready:", DST)


YOLO dataset ready: dataset_yolo_cls


Установка и проверка ultralytics
YOLO от Ultralytics тренируется через from ultralytics import YOLO и поддерживает режим классификации с yolo11n-cls.pt

In [26]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.5 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.243
    Uninstalling ultralytics-8.3.243:
      Successfully uninstalled ultralytics-8.3.243


In [27]:
from ultralytics import YOLO

Количество эпох — это сколько раз модель “пройдёт” по всему train‑набору, но в Ultralytics обычно ставят epochs с запасом и используют `patience` (early stopping), чтобы обучение остановилось, когда метрики на val перестали улучшаться.

## Сколько эпох ставить  (практично)
- Для быстрого чернового прогона: **10–30 эпох** (чтобы проверить, что всё работает и метрики растут).
- Для “нормального” обучения: часто ставят **100–300 эпох**, а остановку доверяют `patience`, чтобы не тратить лишнее время и не уйти в переобучение.
- Если датасет маленький/простой, модель может выйти на плато быстро — тогда `patience` (например 10–20) сработает раньше, и реальные эпохи будут меньше заданных.

## Рекомендованные настройки прямо в коде
1) “Быстро и безопасно”:
```python
results = model.train(
    data="dataset_yolo_cls",
    epochs=50,
    patience=10,   # остановится, если нет улучшения 10 эпох 
    imgsz=224,
    batch=32
)
```

2) “С запасом, пусть сама остановится”:
```python
results = model.train(
    data="dataset_yolo_cls",
    epochs=300,    # частый стартовый ориентир 
    patience=20,
    imgsz=224,
    batch=32
)
```


In [28]:
from ultralytics import YOLO
from pathlib import Path

DST = Path("dataset_yolo_cls")

model = YOLO("yolo11n-cls.pt")  # предобученная модель для классификации 

results = model.train(
    data=str(DST),     # корень, где лежат train/val
    epochs=2,
    imgsz=224,
    batch=32,
    patience=10        # ранняя остановка, если метрика не улучшается
)


Ultralytics 8.3.248 🚀 Python-3.12.6 torch-2.9.1 CPU (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_yolo_cls, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=

In [29]:
val_res = model.val(data=str(DST))
val_res


Ultralytics 8.3.248 🚀 Python-3.12.6 torch-2.9.1 CPU (Apple M3)


YOLO11n-cls summary (fused): 47 layers, 1,538,834 parameters, 0 gradients, 3.2 GFLOPs
train: /Users/veronikadenisenko/Documents/Preparing_Rea2026/Module 1/Изображения/dataset_yolo_cls/train... found 2319 images in 10 classes ✅ 
val: /Users/veronikadenisenko/Documents/Preparing_Rea2026/Module 1/Изображения/dataset_yolo_cls/val... found 194 images in 10 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 157.4±94.7 MB/s, size: 3.6 KB)
val: Scanning /Users/veronikadenisenko/Documents/Preparing_Rea2026/Module 1/Изображения/dataset_yolo_cls/val... 194 images, 0 corrupt: 100% ━━━━━━━━━━━━ 194/194 907.1Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 13/13 5.3it/s 2.4s0.2s
                   all      0.964          1
Speed: 0.0ms preprocess, 10.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /Users/veronikadenisenko/Documents/Preparing_Rea2026/Module 1/Изображения/runs/classify/val4


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x3374c1f40>
curves: []
curves_results: []
fitness: 0.9819587767124176
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9639175534248352, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9819587767124176}
save_dir: PosixPath('/Users/veronikadenisenko/Documents/Preparing_Rea2026/Module 1/Изображения/runs/classify/val4')
speed: {'preprocess': 0.0006806288924447463, 'inference': 10.898079252545303, 'loss': 4.4886620184790686e-05, 'postprocess': 0.00011855677867154639}
task: 'classify'
top1: 0.9639175534248352
top5: 1.0

Что означает каждая метрика:
Accuracy: доля правильных предсказаний (top‑1 попадание).

Top‑5 accuracy: правильный класс в топ‑5 по вероятности (важно при большом числе классов).
​

Precision (macro): “когда модель сказала класс, насколько часто права”, усреднение по классам без учёта размера классов.

Recall (macro): “сколько объектов каждого класса модель нашла”, усреднение по классам.

F1 (macro): баланс precision и recall, честнее при дисбалансе.

Balanced accuracy: средний recall по классам (полезно при дисбалансе).

Confusion matrix: какие классы с какими путаются.

ROC-AUC (ovr): насколько хорошо модель ранжирует правильный класс выше неправильных по вероятностям (нужны вероятности, а не только метки).

In [30]:
import numpy as np
from pathlib import Path

VAL_DIR = Path("dataset_yolo_cls/val")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# список файлов и истинные метки (из папки)
val_paths = []
y_true_names = []

for class_dir in sorted([p for p in VAL_DIR.iterdir() if p.is_dir()]):
    for p in class_dir.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            val_paths.append(str(p))
            y_true_names.append(class_dir.name)

# делаем предсказания батчами (Ultralytics сам батчит)
pred_results = model.predict(source=val_paths, imgsz=224, verbose=False)

# превращаем в id-метки
name_to_id = {v: int(k) for k, v in model.names.items()}  # model.names: id->name 
n_classes = len(model.names)

y_true = np.array([name_to_id[n] for n in y_true_names], dtype=int)

y_pred = []
y_proba = []

for r in pred_results:
    # probs содержит вероятности классов + top1/top5 и confidence 
    probs = r.probs
    y_pred.append(int(probs.top1))                 # id top-1 класса 
    y_proba.append(probs.data.cpu().numpy())       # полный вектор вероятностей 

y_pred = np.array(y_pred, dtype=int)
y_proba = np.stack(y_proba)

print("val samples:", len(y_true), "classes:", n_classes)


val samples: 194 classes: 10


In [31]:
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score,
    precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    top_k_accuracy_score, roc_auc_score
)

print("Accuracy (top-1):", accuracy_score(y_true, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_true, y_pred))
print("Precision macro:", precision_score(y_true, y_pred, average="macro", zero_division=0))
print("Recall macro:", recall_score(y_true, y_pred, average="macro", zero_division=0))
print("F1 macro:", f1_score(y_true, y_pred, average="macro", zero_division=0))
print("F1 weighted:", f1_score(y_true, y_pred, average="weighted", zero_division=0))

print("Top-5 accuracy:", top_k_accuracy_score(y_true, y_proba, k=min(5, y_proba.shape[1])))

# ROC-AUC для мультикласса 
try:
    print("ROC-AUC ovr macro:", roc_auc_score(y_true, y_proba, multi_class="ovr", average="macro"))
    print("ROC-AUC ovr weighted:", roc_auc_score(y_true, y_proba, multi_class="ovr", average="weighted"))
except Exception as e:
    print("ROC-AUC error:", e)

print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred, target_names=[model.names[i] for i in range(n_classes)]))


Accuracy (top-1): 0.9639175257731959
Balanced accuracy: 0.963391812865497
Precision macro: 0.9659038901601831
Recall macro: 0.963391812865497
F1 macro: 0.9637080759724579
F1 weighted: 0.9641120302554517
Top-5 accuracy: 1.0
ROC-AUC ovr macro: 0.998372353951755
ROC-AUC ovr weighted: 0.9983765810561047

Confusion matrix:
 [[20  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  1  0]
 [ 0  0  0  0 20  0  0  0  0  0]
 [ 0  0  0  0  0 17  1  0  0  0]
 [ 0  0  0  0  3  0 16  0  0  0]
 [ 0  0  0  0  0  0  2 18  0  0]
 [ 0  0  0  0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0  0  0  0 20]]

Classification report:
                     precision    recall  f1-score   support

            Beagle       1.00      1.00      1.00        20
             Boxer       1.00      1.00      1.00        20
           Bulldog       1.00      1.00      1.00        20
         Dachshund       1.00      0.95      0.97        19
   German_Shepherd    